### Extract Reviews Directly from the Website (GSM_ARENA)

In [115]:
'''''
import requests
import random
from bs4 import BeautifulSoup
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

# Function to clean text
def clean_text(text):
    # Remove HTML tags
    text = BeautifulSoup(text, "lxml").get_text()
    # Remove non-alphabetic characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Convert to lowercase
    text = text.lower()
    # Tokenize the text
    tokens = word_tokenize(text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]
    # Join the tokens back into text
    cleaned_text = ' '.join(filtered_tokens)
    return cleaned_text

P = []

# List of user agents to rotate
user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:54.0) Gecko/20100101 Firefox/54.0',
    'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36',
    # Add more user agents as needed
]

# Fetch every 10 pages
for i in range(2, 542, 10):
    url = f"https://www.gsmarena.com/apple_iphone_6-reviews-6378p{i}.php"

    # Rotate user agent
    headers = {
        'User-Agent': random.choice(user_agents)
    }

    r = requests.get(url, headers=headers)
    print(f"Fetching page {i}: {r.status_code}")

    if r.status_code != 200:
        print(f"Failed to retrieve page {i}")
        continue

    soup = BeautifulSoup(r.text, "lxml")

    reviews = soup.find_all('div', class_='user-thread')
    if not reviews:
        print(f"No reviews found on page {i}")

    for review in reviews:
        text = review.find('p', class_='uopin')
        if text:
            k = text.get_text(strip=True)
            cleaned_review = clean_text(k)
            P.append(cleaned_review)

# Save the cleaned data to a CSV file
df = pd.DataFrame(P, columns=["Cleaned_Review"])
#df.to_csv("GSM_ARENA_REVIEWS.csv", index=False)
print("Cleaned data saved successfully.")
'''''

'\'\'\nimport requests\nimport random\nfrom bs4 import BeautifulSoup\nimport pandas as pd\nimport re\nimport nltk\nfrom nltk.corpus import stopwords\nfrom nltk.tokenize import word_tokenize\n\n# Download NLTK resources\nnltk.download(\'punkt\')\nnltk.download(\'stopwords\')\n\n# Function to clean text\ndef clean_text(text):\n    # Remove HTML tags\n    text = BeautifulSoup(text, "lxml").get_text()\n    # Remove non-alphabetic characters\n    text = re.sub(r\'[^a-zA-Z\\s]\', \'\', text)\n    # Convert to lowercase\n    text = text.lower()\n    # Tokenize the text\n    tokens = word_tokenize(text)\n    # Remove stopwords\n    stop_words = set(stopwords.words(\'english\'))\n    filtered_tokens = [word for word in tokens if word not in stop_words]\n    # Join the tokens back into text\n    cleaned_text = \' \'.join(filtered_tokens)\n    return cleaned_text\n\nP = []\n\n# List of user agents to rotate\nuser_agents = [\n    \'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTM

### Cleaned reviews are saved under "GSM_ARENA_REVIEWS.csv"

### Import the Cleaned Reviews Dataset

In [116]:
import pandas as pd
import string
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import SMOTE
from collections import Counter

In [117]:
import pandas as pd
df2= pd.read_csv("GSM_ARENA_REVIEWS.csv")
df2

,Cleaned_Review
0,switched nokia g today years iphone great phon...
1,add atleast version update
2,anonymous jun still thinnest iphone ever wonde...
3,still thinnest iphone ever wonder beat
4,hussain jul gets mine soonok dont sit
...,...
1068,robot may reply freezegame im android fan appl...
1069,anonymous may fortune teller think prediction ...
1070,reply freezegame im android fan apple fan youv...
1071,anonymous may person know read well bad sense ...


In [118]:
df2 = pd.DataFrame(df2)

In [119]:
df2.shape

(1073, 1)

In [120]:
df2.duplicated().sum()

0

In [121]:
df2["Cleaned_Review"][2]

'anonymous jun still thinnest iphone ever wonder beatyepstill using backup phone'

In [122]:
l = []

for i in range(0, 10):
    dff = df2["Cleaned_Review"][i]
    l.append(dff)
    print(dff)
    print()


switched nokia g today years iphone great phone thumbs upnow obsolete yes years started counting time found time get new phone

add atleast version update

anonymous jun still thinnest iphone ever wonder beatyepstill using backup phone

still thinnest iphone ever wonder beat

hussain jul gets mine soonok dont sit

anonymous mar needs ios apple cant provide ios due hardware limitations

noif market new phone extreme budget get literally anything else eitherthe phone hasnt updated years app compatibility going get worse phone horrifically unreliableeven extreme apple fanboy get low end android phone youll get nice stuff like app support phone doesnt eat batteries destroy motherboardbesides low end smartphones nowadays plenty powerful enough even bloated social media programs

used ph updated ios time ph wifi network system problem solve problem

last iphone continued ergonomic advantage predecessor instead straight downgrade regard iphone g rounded g making feel better hand iphone feel g

### Perform Sentiment Analysis Using VADER model.

In [123]:
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Download the VADER lexicon
nltk.download('vader_lexicon')

# Load your CSV file

reviews_df = pd.read_csv("GSM_ARENA_REVIEWS.csv")

# Initialize the VADER sentiment intensity analyzer
sid = SentimentIntensityAnalyzer()

# Function to get sentiment from review text
def get_sentiment(review):
    scores = sid.polarity_scores(review)
    if scores['compound'] >= 0.05:
        return 'Positive'
    elif scores['compound'] <= -0.05:
        return 'Negative'
    else:
        return 'Neutral'


# Convert values in 'Cleaned_Review' column to strings
reviews_df['Cleaned_Review'] = reviews_df['Cleaned_Review'].astype(str)

# Apply sentiment analysis to the cleaned reviews
reviews_df['Sentiment'] = reviews_df['Cleaned_Review'].apply(get_sentiment)

#reviews_df.to_csv("VADER_Sentiment_File.csv", index=False)
# Display the results
print(reviews_df.head())


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


                                      Cleaned_Review Sentiment
0  switched nokia g today years iphone great phon...  Positive
1                         add atleast version update   Neutral
2  anonymous jun still thinnest iphone ever wonde...   Neutral
3             still thinnest iphone ever wonder beat   Neutral
4              hussain jul gets mine soonok dont sit   Neutral


### Import the VADER_Sentiment_File

In [124]:
df3 = pd.read_csv("VADER_Sentiment_File.csv")
df3

,Cleaned_Review,Sentiment
0,switched nokia g today years iphone great phon...,Positive
1,add atleast version update,Neutral
2,anonymous jun still thinnest iphone ever wonde...,Neutral
3,still thinnest iphone ever wonder beat,Neutral
4,hussain jul gets mine soonok dont sit,Neutral
...,...,...
1068,robot may reply freezegame im android fan appl...,Positive
1069,anonymous may fortune teller think prediction ...,Positive
1070,reply freezegame im android fan apple fan youv...,Neutral
1071,anonymous may person know read well bad sense ...,Positive


In [125]:
df3.duplicated().sum()

0

In [126]:
df3.head()

,Cleaned_Review,Sentiment
0,switched nokia g today years iphone great phon...,Positive
1,add atleast version update,Neutral
2,anonymous jun still thinnest iphone ever wonde...,Neutral
3,still thinnest iphone ever wonder beat,Neutral
4,hussain jul gets mine soonok dont sit,Neutral


In [127]:
df3.tail()

,Cleaned_Review,Sentiment
1068,robot may reply freezegame im android fan appl...,Positive
1069,anonymous may fortune teller think prediction ...,Positive
1070,reply freezegame im android fan apple fan youv...,Neutral
1071,anonymous may person know read well bad sense ...,Positive
1072,fanboys dreaming iphone even official rumor im...,Neutral


In [128]:
df3[df3["Sentiment"] == "Positive"].count()

Cleaned_Review    687
Sentiment         687
dtype: int64

In [129]:
df3["Sentiment"].value_counts()

Sentiment
Positive    687
Negative    217
Neutral     169
Name: count, dtype: int64

In [130]:
687+217+169

1073

In [131]:
# Define the text and label columns
text_column = 'Cleaned_Review'
label_column = 'Sentiment'

# Download stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Function to preprocess text
def preprocess_text(text):
    if isinstance(text, str):
        # Convert to lowercase
        text = text.lower()
        # Remove punctuation
        text = text.translate(str.maketrans('', '', string.punctuation))
        # Remove stopwords
        text = ' '.join([word for word in text.split() if word not in stop_words])
        return text
    else:
        return ''



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [132]:
# Apply preprocessing to the text column
data[text_column] = data[text_column].apply(preprocess_text)

In [133]:

# Use bi-grams and tri-grams for TF-IDF vectorization
vectorizer = TfidfVectorizer(ngram_range=(1, 3))
X = vectorizer.fit_transform(data[text_column])
y = data[label_column]


In [134]:
# Balance the dataset using SMOTE
print('Original dataset shape %s' % Counter(y))
smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X, y)
print('Resampled dataset shape %s' % Counter(y_res))

Original dataset shape Counter({'Positive': 687, 'Negative': 217, 'Neutral': 169})
Resampled dataset shape Counter({'Positive': 687, 'Neutral': 687, 'Negative': 687})


In [135]:
# Split resampled data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2, random_state=42)


In [136]:
# Hyperparameter tuning with Grid Search for SVM
param_grid = {'C': [0.1, 1, 10, 100], 'kernel': ['linear', 'rbf', 'poly']}
grid_search = GridSearchCV(SVC(), param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)


GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100],
                         'kernel': ['linear', 'rbf', 'poly']},
             scoring='accuracy')

In [137]:
# Get best parameters and evaluate the best model
best_svm = grid_search.best_estimator_
y_pred_svm = best_svm.predict(X_test)
print("SVM Classification Report after Grid Search:")
print(classification_report(y_test, y_pred_svm))
print(f'SVM Accuracy after Grid Search: {accuracy_score(y_test, y_pred_svm)}')


SVM Classification Report after Grid Search:
              precision    recall  f1-score   support

    Negative       0.98      0.96      0.97       138
     Neutral       0.99      1.00      1.00       127
    Positive       0.97      0.98      0.98       148

    accuracy                           0.98       413
   macro avg       0.98      0.98      0.98       413
weighted avg       0.98      0.98      0.98       413

SVM Accuracy after Grid Search: 0.9806295399515739


In [138]:
# Train the Naive Bayes model
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)

# Evaluate the Naive Bayes model
y_pred_nb = nb_model.predict(X_test)
print("Naive Bayes Classification Report:")
print(classification_report(y_test, y_pred_nb))
print(f'Naive Bayes Accuracy: {accuracy_score(y_test, y_pred_nb)}')

Naive Bayes Classification Report:
              precision    recall  f1-score   support

    Negative       0.86      0.97      0.91       138
     Neutral       0.89      0.99      0.94       127
    Positive       1.00      0.78      0.88       148

    accuracy                           0.91       413
   macro avg       0.92      0.92      0.91       413
weighted avg       0.92      0.91      0.91       413

Naive Bayes Accuracy: 0.910411622276029


In [157]:
# Train the Random Forest model
rf_model = RandomForestClassifier(n_estimators=32, random_state=42)
rf_model.fit(X_train, y_train)

# Evaluate the Random Forest model
y_pred_rf = rf_model.predict(X_test)
print("Random Forest Classification Report:")
print(classification_report(y_test, y_pred_rf))
print(f'Random Forest Accuracy: {accuracy_score(y_test, y_pred_rf)}')

Random Forest Classification Report:
              precision    recall  f1-score   support

    Negative       0.98      0.90      0.94       138
     Neutral       0.82      1.00      0.90       127
    Positive       0.95      0.84      0.89       148

    accuracy                           0.91       413
   macro avg       0.91      0.91      0.91       413
weighted avg       0.92      0.91      0.91       413

Random Forest Accuracy: 0.9079903147699758
